In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from tensorflow import keras
from keras.layers.core import Dense
from tensorflow.keras.layers import Input , Flatten
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
from keras.callbacks import ReduceLROnPlateau
from keras.applications.inception_v3 import InceptionV3
%matplotlib inline

print("Setup Complete!")



Setup Complete!


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
pip install split_folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
img_folder='/content/gdrive/MyDrive/Sprint project/Scenes training set'
import splitfolders
splitfolders.ratio(img_folder, output="output",
    seed=1337, ratio=(.7,0.3), group_prefix=None, move=False)

Copying files: 14034 files [03:20, 70.07 files/s] 


In [5]:
train_dir = '/content/output/train'
val_dir = '/content/output/val'
test_dir='/content/gdrive/MyDrive/Sprint project/Testing'

In [6]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen= ImageDataGenerator(rescale=1. /255)
train_generator=train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=30,
    shuffle=False,
    seed=42,
    class_mode='categorical',
    subset = 'training'
)

Found 9820 images belonging to 6 classes.


In [7]:
from keras.preprocessing.image import ImageDataGenerator
val_datagen= ImageDataGenerator(rescale=1. /255)
val_generator= val_datagen.flow_from_directory(
     directory=val_dir,
     target_size=(224, 224),
     color_mode= "rgb",
     batch_size=30,
     shuffle= False,
     seed=42,
)

Found 4214 images belonging to 6 classes.


In [8]:
from keras.preprocessing.image import ImageDataGenerator
test_datagen= ImageDataGenerator(rescale=1. /255)
test_generator=test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=30,
    shuffle=False,
    seed=42)

Found 3050 images belonging to 1 classes.


In [9]:
inceptionV3 = InceptionV3(include_top= False, input_shape=(224,224,3))

for layer in inceptionV3.layers:
    layer.trainable = False

87924736/87910968 [==============================] - 0s 0us/step


In [10]:
last_layer = inceptionV3.get_layer('mixed9')

print('last layer output shape: ', last_layer.output_shape)

last_output = last_layer.output

last layer output shape:  (None, 5, 5, 2048)


In [11]:
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(units = 1024, activation = tf.nn.relu)(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense  (6, activation = tf.nn.softmax)(x)

model = tf.keras.Model( inceptionV3.input, x)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=1,
                                            verbose=1,
                                            factor=0.25,
                                            min_lr=0.000003)

model.compile(loss = 'categorical_crossentropy', optimizer= tf.keras.optimizers.Adam(), metrics=['acc'])

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [12]:
history = model.fit(train_generator,
                    steps_per_epoch =100,
                    epochs = 25,
                   validation_data = val_generator,
                   validation_steps=30
                    )

Epoch 1/25
100/100 [==============================] - 25s 108ms/step - loss: 3.1953 - acc: 0.6890 - val_loss: 0.1035 - val_acc: 0.9767
Epoch 2/25
100/100 [==============================] - 11s 109ms/step - loss: 0.3563 - acc: 0.8661 - val_loss: 0.4902 - val_acc: 0.8344
Epoch 3/25
100/100 [==============================] - 10s 98ms/step - loss: 0.3403 - acc: 0.8673 - val_loss: 0.3266 - val_acc: 0.8956
Epoch 4/25
100/100 [==============================] - 10s 98ms/step - loss: 0.2170 - acc: 0.9153 - val_loss: 0.3366 - val_acc: 0.9033
Epoch 5/25
100/100 [==============================] - 10s 98ms/step - loss: 0.2068 - acc: 0.9243 - val_loss: 0.2274 - val_acc: 0.9267
Epoch 6/25
100/100 [==============================] - 10s 99ms/step - loss: 0.1562 - acc: 0.9433 - val_loss: 0.2883 - val_acc: 0.9178
Epoch 7/25
100/100 [==============================] - 10s 99ms/step - loss: 0.1481 - acc: 0.9473 - val_loss: 0.1172 - val_acc: 0.9567
Epoch 8/25
100/100 [==============================] - 10s 99

In [13]:
model.evaluate_generator(generator=train_generator,steps=100)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


[0.0039984602481126785, 0.9990000128746033]

In [14]:
model.evaluate_generator(generator=val_generator,steps=30)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


[0.30599355697631836, 0.9411110877990723]

In [18]:
import os 
import glob
import pandas as pd

Name = []

for img in glob.glob(test_dir):
    prediction = model.predict(test_generator,steps=3050,verbose=1)
    for name in os.listdir('/content/gdrive/MyDrive/Sprint project/Testing/Scenes testing test'):
         Name.append(name[0::])



predicted_class_indics=np.argmax(prediction,axis=1)
Name.sort()
Temp = {'Image':Name, 'Label':predicted_class_indics}
temp = pd.DataFrame(Temp)
temp.to_csv('/content/result.csv', index=False)

 101/3050 [..............................] - ETA: 4:38

3050/3050 [==============================] - 10s 3ms/step


In [19]:
temp

,Image,Label
0,1020.jpg,0
1,103.jpg,0
2,1053.jpg,0
3,1081.jpg,0
4,1082.jpg,0
...,...,...
3045,920.jpg,0
3046,949.jpg,0
3047,952.jpg,0
3048,961.jpg,0
